# Used car's price prediction


Необходимо разработать модель для предсказания цены автомобиля на вторичном рынке

# Исследование данных

Импортируем все необходимые библиотеки

In [32]:
import pandas as pd
import pandas_profiling
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error
from catboost import CatBoostRegressor
import os
import catboost
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from datetime import datetime
from sklearn.impute import SimpleImputer
from sklearn.ensemble import VotingRegressor
import joblib

Загрузим обучающую и тестовую выборку, изучим их

In [33]:
data_train=pd.read_csv('train.csv')
data_test=pd.read_csv('test.csv')
original=pd.read_csv('sample_submission.csv',index_col=0)

In [34]:
def info(data):
    print(data.head(5))
    print()
    print()
    print(data.info())

In [35]:
info(data_train)

   year    make   model    trim   body transmission                vin state  \
0  2011    Ford    Edge     SEL    suv    automatic  2fmdk3jc4bba41556    md   
1  2014    Ford  Fusion      SE  Sedan    automatic  3fa6p0h75er208976    mo   
2  2012  Nissan  Sentra  2.0 SL  sedan    automatic  3n1ab6ap4cl698412    nj   
3  2003  HUMMER      H2    Base    suv    automatic  5grgn23u93h101360    tx   
4  2007    Ford  Fusion     SEL  Sedan    automatic  3fahp08z17r268380    md   

   condition  odometer  color interior                      seller  \
0        4.2  111041.0  black    black          santander consumer   
1        3.5   31034.0  black    black       ars/avis budget group   
2        2.2   35619.0  black    black          nissan-infiniti lt   
3        2.8  131301.0   gold    beige  wichita falls ford lin inc   
4        2.0  127709.0  black    black                purple heart   

   sellingprice                                 saledate  
0         12500  Tue Jun 02 2015 02:30:

В обучающей выборке  440236 строчек, 15 признаков, из которых 4 числовые и 11 категориальные, целевой признак - "sellingprice"

In [36]:
info(data_test)

   year       make         model      trim      body transmission  \
0  2005   Cadillac           CTS      Base     Sedan    automatic   
1  2014        GMC  Savana Cargo      2500       Van          NaN   
2  2013     Nissan        Murano         S       SUV    automatic   
3  2013  Chevrolet        Impala  LS Fleet     Sedan    automatic   
4  2013     Nissan         Titan        SV  Crew Cab    automatic   

                 vin state  condition  odometer   color interior  \
0  1g6dp567450124779    ca        2.7  116970.0  silver    black   
1  1gtw7fca7e1902207    pa        4.4    6286.0   white     gray   
2  jn8az1mw6dw303497    oh        4.6   11831.0    gray    black   
3  2g1wf5e34d1160703    fl        2.3   57105.0  silver    black   
4  1n6aa0ec3dn301209    tn        2.9   31083.0   black    black   

                                             seller  \
0                            lexus of stevens creek   
1                                            u-haul   
2          

В тестовой выборке 110060 строк, 14 признаков, из которых 3 числовые и 11 категориальные

# Обработка данных

Общий анализ данных 

In [37]:
pandas_profiling.ProfileReport(data_train)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Изучим данные на явные дубликаты

In [38]:
data_train.duplicated().sum()

0

In [39]:
data_test.duplicated().sum()

0

Явных дубликатов не обнаружено

Найдем количество пропущенных значений в данных

In [40]:
data_train.isna().sum()

year                0
make             8043
model            8123
trim             8337
body            10393
transmission    51461
vin                 0
state               0
condition        9405
odometer           69
color             586
interior          586
seller              0
sellingprice        0
saledate            0
dtype: int64

Большое количество пропущенных значений обнаружено в колонке "transmission", также пропущенные значения есть в колонках "make","model","trim","body","condition","odometer","color","interior"

In [41]:
data_test.isna().sum()

year                0
make             2061
model            2079
trim             2114
body             2594
transmission    13012
vin                 0
state               0
condition        2379
odometer           19
color             158
interior          158
seller              0
saledate            0
dtype: int64

Аналогичная ситуация в тестовой выборке

Посмотрим на распределние данных в числовых признаках

In [42]:
data_train.hist()

array([[<AxesSubplot:title={'center':'year'}>,
        <AxesSubplot:title={'center':'condition'}>],
       [<AxesSubplot:title={'center':'odometer'}>,
        <AxesSubplot:title={'center':'sellingprice'}>]], dtype=object)

Аномальных данных не обнаружено в численных признаках

In [43]:
data_test.hist()

array([[<AxesSubplot:title={'center':'year'}>,
        <AxesSubplot:title={'center':'condition'}>],
       [<AxesSubplot:title={'center':'odometer'}>, <AxesSubplot:>]],
      dtype=object)

Приведем столбец "saledate" к формату даты 

In [44]:
data_train['saledate']=data_train['saledate'].replace('PST','',regex=True)
data_train['saledate']=data_train['saledate'].replace('PDT','',regex=True)
data_train['saledate']=[date_str.replace('()', '') for date_str in data_train['saledate']]

In [45]:
data_test['saledate']=data_test['saledate'].replace('PST','',regex=True)
data_test['saledate']=data_test['saledate'].replace('PDT','',regex=True)
data_test['saledate']=[date_str.replace('()', '') for date_str in data_test['saledate']]

In [46]:
data_train['saledate'] = pd.to_datetime(data_train['saledate'], format='%a %b %d %Y %H:%M:%S GMT%z ',utc=True)
data_test['saledate'] = pd.to_datetime(data_test['saledate'], format='%a %b %d %Y %H:%M:%S GMT%z ',utc=True)

Выделим только год покупки автомобиля

In [47]:
data_train['saledate'] = pd.to_datetime(data_train['saledate'])
data_train['saledate'] = data_train['saledate'].dt.year

In [48]:
data_test['saledate'] = pd.to_datetime(data_test['saledate'])
data_test['saledate'] = data_test['saledate'].dt.year

In [49]:
data_test['saledate'].unique()

array([2015, 2014])

Создадим новый признак - возраст машины( разница между годом продажы и годом выпуска) и удалим столбцы год выпуска и дата продажи, потому что важна только разница между ними

In [50]:
data_train['age_car']=data_train['saledate']-data_train['year']
data_test['age_car']=data_test['saledate']-data_test['year']

In [51]:
data_train=data_train.drop(['saledate','year'],axis=1)
data_test=data_test.drop(['saledate','year'],axis=1)

Перейдем к поиску неявных дубликатов в категориальных признаках

Категориальные признаки будем обрабатывать по следующему алгоритму:
1. Приводим данные к нижнему регистру
2. Заменяем пропущенные значения на "other"

In [52]:
data_train['make'].unique()

array(['Ford', 'Nissan', 'HUMMER', 'Lincoln', 'pontiac', 'Hyundai', nan,
       'Buick', 'Chevrolet', 'Honda', 'Acura', 'Cadillac', 'GMC', 'Saab',
       'Dodge', 'Mercedes-Benz', 'Toyota', 'Volkswagen', 'BMW',
       'Infiniti', 'Chrysler', 'Kia', 'Jaguar', 'Subaru', 'Jeep', 'Lexus',
       'mercedes', 'Scion', 'FIAT', 'Suzuki', 'Mazda', 'Volvo', 'Audi',
       'MINI', 'Isuzu', 'Mitsubishi', 'smart', 'Pontiac', 'Porsche',
       'subaru', 'ford', 'Land Rover', 'chrysler', 'Saturn', 'mazda',
       'dodge', 'Ram', 'Oldsmobile', 'hyundai', 'Mercury', 'Bentley',
       'toyota', 'lincoln', 'Fisker', 'nissan', 'chevrolet', 'honda',
       'porsche', 'mitsubishi', 'lexus', 'bmw', 'Maserati', 'acura',
       'jeep', 'mercury', 'Tesla', 'landrover', 'vw', 'cadillac', 'buick',
       'gmc truck', 'land rover', 'volkswagen', 'Rolls-Royce', 'audi',
       'Ferrari', 'suzuki', 'Plymouth', 'oldsmobile', 'Lamborghini',
       'gmc', 'Geo', 'ford truck', 'Aston Martin', 'plymouth', 'Daewoo',
      

Приведем данные к нижнему регистру и в ручную обработаем дубликаты в столбце "make"

In [53]:
data_train['make']=data_train['make'].str.lower()
data_test['make']=data_test['make'].str.lower()

In [54]:
data_train['make']=data_train['make'].replace('mercedes-benz','mercedes',regex=True)
data_train['make']=data_train['make'].replace('mercedes-b','mercedes',regex=True)
data_train['make']=data_train['make'].replace('mercedes-b','mercedes',regex=True)
data_train['make']=data_train['make'].replace('vw','volkswagen',regex=True)
data_train['make']=data_train['make'].replace('mazda tk','mazda',regex=True)
data_train['make']=data_train['make'].replace('dodge tk','dodge',regex=True)
data_train['make']=data_train['make'].replace('dot','dodge',regex=True)
data_train['make']=data_train['make'].replace('airstream','mercedes',regex=True)
data_train['make']=data_train['make'].replace('gmc truck','gmc',regex=True)
data_train['make']=data_train['make'].replace('landrover','land rover',regex=True)
data_train['make']=data_train['make'].fillna('other')

In [55]:
data_test['make']=data_test['make'].replace('mercedes-benz','mercedes',regex=True)
data_test['make']=data_test['make'].replace('mercedes-b','mercedes',regex=True)
data_test['make']=data_test['make'].replace('mercedes-b','mercedes',regex=True)
data_test['make']=data_test['make'].replace('vw','volkswagen',regex=True)
data_test['make']=data_test['make'].replace('mazda tk','mazda',regex=True)
data_test['make']=data_test['make'].replace('dodge tk','dodge',regex=True)
data_test['make']=data_test['make'].replace('dot','dodge',regex=True)
data_test['make']=data_test['make'].replace('airstream','mercedes',regex=True)
data_test['make']=data_test['make'].replace('gmc truck','gmc',regex=True)
data_test['make']=data_test['make'].replace('landrover','land rover',regex=True)
data_test['make']=data_test['make'].fillna('other')

In [56]:
data_train['model'].unique()

array(['Edge', 'Fusion', 'Sentra', 'H2', 'MKZ', 'g6', 'Escape', 'Elantra',
       'Freestyle', nan, 'Lucerne', 'Windstar', 'Silverado 1500',
       'Murano', 'Equinox', 'Accord', 'Civic', 'MDX', 'CTS', 'Taurus',
       'SRX', 'Yukon', '9-3', 'Explorer', 'F-150', 'Charger', 'Armada',
       'GL-Class', 'Avalon', 'Tahoe', 'Malibu', 'Passat', 'Camry', 'Flex',
       '3 Series', 'Q50', 'Altima', 'Mustang', '300', 'Sonata',
       'Envoy XL', 'Rogue', 'Accent', 'Maxima', 'Forte', 'XF', 'Outback',
       'Grand Caravan', 'E-Class', 'Cherokee', 'Impala', 'Optima',
       '5 Series', 'Expedition', 'IS 250', 'MKS', '200', 'Veloster',
       'Golf', 'RAV4', 'e300dt', 'Five Hundred', 'G Coupe',
       'Grand Cherokee', 'X-Type', 'G Convertible', 'Tacoma', 'xA',
       'G Sedan', 'TL', 'Liberty', 'Soul', '500L', 'Town and Country',
       'HHR', 'Wrangler', 'Reno', 'Suburban', 'PT Cruiser', 'GX 460',
       'Sienna', 'SL-Class', 'Envoy', 'Patriot', 'E-Series Van', 'Versa',
       'Aspen', 'M', 'Co

In [57]:
data_train['model']=data_train['model'].str.lower()
data_train['model']=data_train['model'].fillna('other')

In [58]:
data_test['model']=data_test['model'].str.lower()
data_test['model']=data_test['model'].fillna('other')

In [59]:
data_train['trim'].unique()

array(['SEL', 'SE', '2.0 SL', ..., '2.5 X L.L.Bean Edition',
       '3500 High Roof 140 WB', '4x4 v6 xlt sport'], dtype=object)

In [60]:
data_train['trim']=data_train['trim'].str.lower()
data_train['trim']=data_train['trim'].fillna('other')

In [61]:
data_test['trim']=data_test['trim'].str.lower()
data_test['trim']=data_test['trim'].fillna('other')

In [62]:
data_train['body'].unique()

array(['suv', 'Sedan', 'sedan', nan, 'SUV', 'wagon', 'Minivan',
       'Extended Cab', 'Regular Cab', 'Coupe', 'SuperCrew', 'Wagon',
       'convertible', 'Crew Cab', 'SuperCab', 'Convertible', 'Hatchback',
       'minivan', 'hatchback', 'G Coupe', 'G Convertible', 'coupe',
       'Access Cab', 'G Sedan', 'regular cab', 'e-series van',
       'supercrew', 'Quad Cab', 'tsx sport wagon', 'Van', 'g sedan',
       'E-Series Van', 'CTS Coupe', 'Koup', 'King Cab', 'extended cab',
       'double cab', 'Elantra Coupe', 'koup', 'access cab', 'Double Cab',
       'crew cab', 'quad cab', 'g coupe', 'CrewMax Cab', 'supercab',
       'g convertible', 'Genesis Coupe', 'van', 'G37 Coupe', 'club cab',
       'Beetle Convertible', 'Mega Cab', 'regular-cab', 'Xtracab',
       'cts coupe', 'genesis coupe', 'Club Cab', 'q60 coupe', 'mega cab',
       'crewmax cab', 'Promaster Cargo Van', 'king cab', 'CTS-V Coupe',
       'TSX Sport Wagon', 'CTS Wagon', 'Cab Plus 4', 'G37 Convertible',
       'Transit Van'

In [63]:
data_train['body']=data_train['body'].str.lower()
data_train['body']=data_train['body'].fillna('other')

In [64]:
data_test['body']=data_test['body'].str.lower()
data_test['body']=data_test['body'].fillna('other')

In [65]:
data_train['color'].unique()

array(['black', 'gold', 'silver', 'blue', 'white', 'gray', '—', 'red',
       'brown', 'green', 'beige', 'orange', nan, 'off-white', 'burgundy',
       'yellow', 'charcoal', 'purple', 'turquoise', 'lime', 'pink'],
      dtype=object)

In [66]:
data_train['color']=data_train['color'].replace('—','other',regex=True)
data_train['color']=data_train['color'].fillna('other')

In [67]:
data_test['color']=data_test['color'].replace('—','other',regex=True)
data_test['color']=data_test['color'].fillna('other')

In [68]:
data_train['interior'].unique()

array(['black', 'beige', 'gray', 'tan', 'brown', '—', 'off-white', nan,
       'blue', 'white', 'silver', 'red', 'green', 'gold', 'purple',
       'orange', 'burgundy', 'yellow'], dtype=object)

In [69]:
data_train['interior']=data_train['interior'].replace('—','other',regex=True)
data_train['interior']=data_train['interior'].fillna('other')

In [70]:
data_test['interior']=data_test['interior'].replace('—','other',regex=True)
data_test['interior']=data_test['interior'].fillna('other')

In [71]:
data_train['seller'].unique()

array(['santander consumer', 'ars/avis budget group',
       'nissan-infiniti lt', ..., 'autostar enterprises',
       'kocourek nissan', 'studio city auto group'], dtype=object)

In [72]:
data_train['seller']=data_train['seller'].str.lower()
data_train['seller']=data_train['seller'].fillna('other')

In [73]:
data_test['seller']=data_test['seller'].str.lower()
data_test['seller']=data_test['seller'].fillna('other')

In [74]:
data_train['transmission'].unique()

array(['automatic', nan, 'manual'], dtype=object)

In [75]:
data_train['transmission']=data_train['transmission'].fillna('other')

In [76]:
data_test['transmission']=data_test['transmission'].fillna('other')

Все категориальные признаки обработаны, теперь перейдем к обработке числовых признаков

Пропуски в числовых признаках будем заменять средним значением

In [123]:
plt.hist(data_train['condition'])

(array([ 6071.,   556., 57147., 28456., 52075., 45683., 83814., 49528.,
        68744., 48162.]),
 array([1. , 1.4, 1.8, 2.2, 2.6, 3. , 3.4, 3.8, 4.2, 4.6, 5. ]),
 <BarContainer object of 10 artists>)

In [78]:
data_train['condition']=data_train['condition'].fillna(data_train['condition'].mean())
data_test['condition']=data_test['condition'].fillna(data_test['condition'].mean())

Заменили пропуски в колонке 'condition' средним значением

In [79]:
plt.hist(data_train['odometer'])

(array([3.31694e+05, 9.87750e+04, 9.11200e+03, 4.94000e+02, 3.00000e+01,
        3.00000e+00, 2.00000e+00, 0.00000e+00, 0.00000e+00, 5.70000e+01]),
 array([1.000000e+00, 1.000008e+05, 2.000006e+05, 3.000004e+05,
        4.000002e+05, 5.000000e+05, 5.999998e+05, 6.999996e+05,
        7.999994e+05, 8.999992e+05, 9.999990e+05]),
 <BarContainer object of 10 artists>)

In [80]:
data_train['odometer']=data_train['odometer'].fillna(data_train['odometer'].mean())
data_test['odometer']=data_test['odometer'].fillna(data_test['odometer'].mean())

In [81]:
data_train.isna().sum()

make            0
model           0
trim            0
body            0
transmission    0
vin             0
state           0
condition       0
odometer        0
color           0
interior        0
seller          0
sellingprice    0
age_car         0
dtype: int64

In [82]:
data_test.isna().sum()

make            0
model           0
trim            0
body            0
transmission    0
vin             0
state           0
condition       0
odometer        0
color           0
interior        0
seller          0
age_car         0
dtype: int64

Все пропуски обработаны

# Разработка модели ML

Перейдем к разработке самой модели, для начала напишем функцию для кодирования категориальных признаков, воспользуемся OE , так как она предпочтительнее для деревьев, в качестве метрики качества возьмем MAPE

In [94]:
def encode(data):
    data_train=data
    encoder = OrdinalEncoder()
    columns=['make','model','trim','body','transmission','vin','state','color','interior','seller']
    cat_train_oe=pd.DataFrame(encoder.fit_transform(data_train[columns]),
    columns=columns)
    dat=data_train.drop(columns,axis=1)
    
    data_train_oe=pd.concat([dat,cat_train_oe],axis=1)
    return(data_train_oe)

Первый метод, которым воспользуемся - линейная регрессия , перед обучением модели пользуемся методом StandardScaler, чтобы масштабировать признаки

In [95]:
def linear(data_train):

    data_train=encode(data_train)
    features=data_train.drop('sellingprice',axis=1)
    target=data_train['sellingprice']
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, 
                                                                              test_size=0.25, 
                                                                              random_state=12345)
    scaler=StandardScaler()
    scaler.fit(features_train)
    features_train=scaler.transform(features_train)
    features_valid=scaler.transform(features_valid)
    
    
    LR=LinearRegression()
    LR.fit(features_train,target_train)
    predictions=LR.predict(features_valid)
    predictions=pd.Series(predictions)
    

    MAPE=mean_absolute_percentage_error(target_valid, predictions)
    return(MAPE)

In [99]:
100*linear(data_train)

78.73399531893715

MAPE на линейной регрессии - 78,73 %

Решающее дерево

In [106]:
def tree(data_train):
    
    data_train=encode(data_train)

    features=data_train.drop(['sellingprice','vin'],axis=1)
    target=data_train['sellingprice']
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, 
                                                                              test_size=0.25, 
                                                                              random_state=12345)
 
    parameters={'max_depth':range(1,40,2),}
    DS=DecisionTreeRegressor(random_state=12345)
    DV=GridSearchCV(DS, parameters,scoring='neg_mean_absolute_percentage_error',cv=5)
    DV.fit(features_train,target_train)
    predictions=pd.Series(DV.predict(features_valid))
    MAPE=mean_absolute_percentage_error(predictions,target_valid)
    
    return(MAPE)
    
    
    
    

In [108]:
100*tree(data_train)

21.631380887221603

MAPE у решающего дерева - 21,6 %

Перейдем к Catboost

In [113]:
def catboost(data_train,data_test):
    features=data_train.drop(['sellingprice','vin'],axis=1)
    columns=['make','model','trim','body','transmission','state','color','interior','seller']
    param_grid = {
        'learning_rate': [0.01, 0.1, 0.5],
        'depth': [4, 6, 8],
        'l2_leaf_reg': [1, 3, 5]
    }
    data_test=data_test.drop(['vin'],axis=1)
    target=data_train['sellingprice']
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, 
                                                                              test_size=0.25, 
                                                                              random_state=12345)
    cat = CatBoostRegressor(random_state=12345,loss_function='RMSE',eval_metric = "MAPE",
leaf_estimation_iterations=30,iterations=3000,use_best_model=True,max_depth=10,early_stopping_rounds=300)
    cat.fit(features_train,target_train,verbose=False,cat_features=columns, plot=True,eval_set=(features_valid,target_valid))
    predictions=cat.predict(data_test)
    return(cat,predictions)

In [118]:
cat,predict=catboost(data_train,data_test)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

MAPE у Catboost - 14,67 %,именно на catboost была обучена лучшая модель

Сохраняем данные для сабмита

In [120]:
pre=pd.Series(predict,index=data_test['vin'])

Сохраняем данные в файл

In [121]:
pre.to_csv('Submit.csv')

Сохраняем полученную модель

In [119]:
joblib.dump(cat, "model.pkl")

['model.pkl']

Оценка важности признаков при обучении модели

In [124]:
importance = pd.DataFrame({
        'Feature': data_train.drop(['sellingprice','vin'],axis=1).columns,
        'Catboost': cat.feature_importances_
    })
importance = importance.sort_values(by="Catboost", ascending=False)

In [129]:
importance

,Feature,Catboost
0,make,19.978330
1,model,15.897038
7,odometer,15.106280
11,age_car,14.900623
3,body,12.043070
2,trim,8.312452
6,condition,5.096738
10,seller,3.407521
5,state,1.936630
9,interior,1.719170


Признаки, имеющие наибольший вес при обучении модели, - "make","model" и "odometr"

# Вывод

Данный проект был реализован в ходе соревнований на платформе Kaggle, данные сначала были загружены и изучены, далее началась работа с пропущенными значениями и поиском неявных дубликатов. После обработки данных можно было приступить к построению самих моделей , первая модель(линейная регрессия) показала не очень хороший результат, MAPE у нее равнялся 78,73 %, вторая модель( решающее дерево) показало результат уже получше,MAPE = 21,6 %, две данные модели были оставлены в финальном блокноте лишь для сравнения , третья и самая лучшая модель - catboost показала результат в 14,67 %, именно эта модель дала лучший результат в соревновании и позволила занять 6 место.Признаки, имеющие наибольший вес при обучении модели, - "make","model" и "odometr". Данная модель была сохранена и далее с помощью streamlit она была упакована в красивый интерфейс для предсказания цены на автомобиль